# Deep Neural Network for MNIST Classifaication

This project is the basic of machine learning also called as the "Hello World" of machine learning. The dataset is called MNIST and refers to handwritten digit recognition. The dataset comprises of 28x28 images of handwritten digits(1 per image) and the goal is to write an algorithm that which digit is written. We will build a network of two hidden layers between inputs and outputs.

In this project i will be using tensorflow as it is the amazing thing when it comes to machine learning. With hust few lines of code we can do a lot.

## Importing relevant packages

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Outlining the model

In [2]:
# Setting the size of the input, output, hidden layer
# Input_size=784 because image dimension is 28x28 which is 784, Output_size=10 because there ar ten digits,
# Setting the hidden_layer on our own as it is a hyperparameter.
input_size=784
output_size=10
hidden_layer_size=50

# Erasing the old memory
tf.reset_default_graph()

# Setting up placeholders for input and target
inputs=tf.placeholder(tf.float32,[None, input_size])
targets=tf.placeholder(tf.float32,[None,output_size])

# Now setting the weights and biases from the first input layer to the hidden layer with the given dimensions
weights_1=tf.get_variable("weights_1",[input_size,hidden_layer_size])
biases_1=tf.get_variable("biases_1",[hidden_layer_size])

#output folows the linear model with the relu function.
outputs_1=tf.nn.relu(tf.matmul(inputs,weights_1)+biases_1)

# Setting the weights and biases from the 1st hidden layer to the 2nd hidden layer followed by the output
weights_2=tf.get_variable("weights_2",[hidden_layer_size,hidden_layer_size])
biases_2=tf.get_variable("biases_2",[hidden_layer_size])

outputs_2=tf.nn.relu(tf.matmul(outputs_1,weights_2)+biases_2)

#Setting the weights and biases from the 2nt hidden layer to the output without the activation function as we have no hidden layer
# left and we have reached the end
weights_3=tf.get_variable("weights_3",[hidden_layer_size,output_size])
biases_3=tf.get_variable("biases_3",[output_size])

outputs=tf.matmul(outputs_2,weights_3)+biases_3

#We apply softmax activation and calculate the cross entropy loss
#Obtaining very small number jeopardizes our model if we don't employ this function
#Logits means unscaled probability. Softmax fits it into a probability distribution
loss=tf.nn.softmax_cross_entropy_with_logits(logits=outputs,labels=targets)

#Finding the mean loss
mean_loss=tf.reduce_mean(loss)

#Setting up the optimizer with the learning rate that minimizes the mean loss
#Using the Adam Optimizer
optimize=tf.train.AdamOptimizer(learning_rate=0.01).minimize(mean_loss)

#Predicting accuracy tf.equal returns bool,argmax return the highest indices in the column
#Checking whether the predicted output by our model and the target output are same or not ie..1/0
out_equals_target=tf.equal(tf.argmax(outputs,1),tf.argmax(targets,1))

#Calculating the accuracy
accuracy=tf.reduce_mean(tf.cast(out_equals_target,tf.float32))

#Session starts
sess=tf.InteractiveSession()

#Initializing all the variables. Tensorflow takes care of the initialization.
initializer=tf.global_variables_initializer()

sess.run(initializer)

# Setting the batch size to 100.This is a hyperparameter
batch_size=100

#Batch number will be total_sample/batch_size
batches_number=mnist.train._num_examples // batch_size

#Setting up th epoch size.This is also a hyperparameter. We can set it to whatever value we can
max_epochs=15
#Setting the pre_validation_loss to be a large value
prev_validation_loss=9999999

#Now the iteration starts
for epoch_counter in range(max_epochs):
    #curr_epoch_loss=0 for every epoch
    curr_epoch_loss=0.
    #Now we will calculate the epoch loss and the validation loss for each and every batch
    #While optimizing we will go front and back and we will keep on optimizing that is reducing the epoch loss
    for batch_counter in range(batches_number):
        
        input_batch, target_batch = mnist.train.next_batch(batch_size)
        
        _,batch_loss = sess.run([optimize,mean_loss],feed_dict={inputs: input_batch,targets: target_batch})
        
        curr_epoch_loss +=batch_loss
        
    curr_epoch_loss/=batches_number
    
    #Now taking the validation data into account we will calculate the validation loss
    #Validation loss means going in forward direction
    input_batch, target_batch=mnist.validation.next_batch(mnist.validation._num_examples)
    
    validation_loss, validation_accuracy=sess.run([mean_loss, accuracy],feed_dict={inputs: input_batch, targets: target_batch})
    
    #Printing the losses and accuracy after each epoch.
    print('Epoch '+str(epoch_counter+1)+
         '.Training loss: '+'{0:.3f}'.format(curr_epoch_loss)+
         '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
         '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.)+'%')
    
    #This is very important as we have to keep track whether overfitting is happening or not
    #Both the training loss and the validation loss decreases usually but as the validation loss increases it is a signal
    #for us that overfitting is going to happen and we should stop training our model.
    if validation_loss > prev_validation_loss:
        break
    prev_validation_loss=validation_loss
    
print('End of training')    

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch 1.Training loss: 0.264. Validation loss: 0.160. Validation accuracy: 94.90%
Epoch 2.Training loss: 0.144. Validation loss: 0.152. Validation accuracy: 95.54%
Epoch 3.Training loss: 0.123. Validation loss: 0.111. Validation accuracy: 96.84%
Epoch 4.Training loss: 0.109. Validation loss: 0.129. Validation accuracy: 96.72%
End of training


## Test

In [3]:
#Now we will be testing the model with few lines code and printing th accuracy value
input_batch, target_batch = mnist.test.next_batch(mnist.test._num_examples)
test_accuracy = sess.run([accuracy],feed_dict={inputs: input_batch, targets: target_batch})

test_accuracy_percent = test_accuracy[0] * 100.

print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%')

Test accuracy: 96.37%
